In [1]:
from nlp import load_dataset
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

In [2]:
train_dataset, test_dataset = load_dataset('emo_data.py', split = ['train', 'test'])

In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 4)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
def tokenize(batch):
    return tokenizer(batch['text'], padding = True, truncation = True)

In [5]:
train_dataset = train_dataset.map(tokenize, batched = True, batch_size = len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched = True, batch_size = len(train_dataset))
train_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }